In [ ]:
import os
import json
import pandas as pd
import traceback

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
OPEN_AI_KEY = os.getenv("OPEN_AI_KEY")
print(OPEN_AI_KEY)

In [ ]:
llm = ChatOpenAI(openai_api_key=OPEN_AI_KEY, model_name="gpt-3.5-turbo", temperature=0.5)
llm

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [ ]:
RESPONSE_JSON = {
  "1": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer"
  },
  "2": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer"
  },
  "3": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here",
    },
    "correct": "correct answer"
  }
}

In [ ]:
PROMPT_TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to 
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
  input_variables=["text", "number", "subject", "tone", "response_json"],
  template=PROMPT_TEMPLATE
)

In [ ]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [ ]:
# verify the qustion and answers generated by AI

EVALUATION_PROMPT_TEMPLATE = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity if the quiz is not at per with the cognitive and analytical abilities of the students,
update the quiz questions which needs to be change the tone such that it perfectly fits the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz.
"""

In [ ]:
quiz_evaluation_prompt = PromptTemplate(
  input_variables=["subject", "quiz"],
  template=EVALUATION_PROMPT_TEMPLATE
)

In [ ]:
quiz_evaluation_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
# Connect the both chain to get the actual output using Sequential Chain

generate_evaluate_chains = SequentialChain(chains=[quiz_chain, quiz_evaluation_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [ ]:
file_path = "../data.txt"

with open(file_path, 'r') as file:
  TEXT = file.read()

In [ ]:
TEXT

In [ ]:
# Serialize the python dictionary into a JSON formatted string
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER = 5
SUBJECT = "biology"
TONE = "simple"

In [ ]:
# get_openai_callback is so important to setup token usage tracking in LangChain

with get_openai_callback() as cb:
  response = generate_evaluate_chains(
    {
      "text": TEXT, 
      "number": NUMBER,
      "subject": SUBJECT,
      "tone": TONE,
      "response_json": json.dumps(RESPONSE_JSON)
    }
  )

In [ ]:
print(f"Total tokens: {cb.total_tokens}")
print(f"Prompt tokens: {cb.prompt_tokens}")
print(f"Completion tokens: {cb.completion_tokens}")
print(f"Total cost: ${cb.total_cost}")

In [ ]:
response

In [ ]:
response_quiz = response.get('quiz')

In [ ]:
response_quiz =json.loads(response_quiz)

In [ ]:
quiz_table_data = []
for key, value in response_quiz.items():
  mcq = value["mcq"]
  options = " | ".join(
    [
      f"{option}: {option_value}" for option, option_value in value["options"].items()
    ]
  )
  correct = value["correct"]
  quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

quiz_table_data

In [ ]:
quiz_df = pd.DataFrame(quiz_table_data)

In [ ]:
quiz_df.to_csv("generated_mcq.csv", index=False)